In [1]:
import re
import os
import json
import gzip
import requests
import random
import logging

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from copy import deepcopy
from time import sleep
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map

In [2]:
proxy_list_url = 'https://www.ip-adress.com/proxy-list'

In [3]:
proxy_df = pd.read_html(proxy_list_url, skiprows=0)[0]

In [4]:
proxy_df.head()

,Proxy,Type,Country,Last Checked
0,52.179.18.244:8080,transparent,United States,47 minutes ago
1,161.117.251.194:80,highly-anonymous,Singapore,57 minutes ago
2,195.123.212.199:44394,highly-anonymous,Latvia,6 hours ago
3,34.125.187.121:3128,transparent,United States,15 hours ago
4,181.118.167.104:80,highly-anonymous,Chile,24 hours ago


In [5]:
proxy_list = proxy_df['Proxy'].values

In [6]:
def get_url_content(url, proxy=[]):
    if len(proxy) > 0:
        proxy_list = list(deepcopy(proxy))
        try:
            random_proxy_server = np.random.choice(proxy_list)
            proxies = {"http": f"http://{random_proxy_server}"}
            return requests.get(url, proxies=proxies, headers={'User-Agent': 'Chrome'}).content
        except Exception as e:
            logging.warning('Failed to use {} proxy. Trying to use another one.....')
            proxy_list.remove(random_proxy_server)
            get_url_content(url, proxy=proxy_list)
    else:
        return requests.get(url, headers={'User-Agent': 'Chrome'}).content

In [7]:
goroh_url = "https://goroh.pp.ua/Транскрипція/{word}"

def parse_goroh(word):
    phonemes = ''  

    try:
        html = get_url_content(goroh_url.format(word=word), proxy=proxy_list)
        soup = BeautifulSoup(html)
        tags = set(filter(None, [item.string.strip() for item in soup.findAll("div", class_="taglist")[0]]))
        if set(["іменник", "прикметник"]).intersection(tags):
            table = soup.findAll("table", class_="table")[0].findAll("tr", class_="row")
            table_content = [[cell.contents[0] for cell in row.findAll("td")] for row in table]
            cell_content = list(filter(lambda x: x[0] == 'називний', table_content))[0][1]
            phonemes = re.findall(r'\>\[(.*)\]\<\/span\>', str(cell_content))[0]
        elif set(["прислівник", "дієслово"]).intersection(tags):
            tag_content = soup.findAll("div", class_="page__sub-header")[0]#.findAll('h2')[0]
            phonemes = re.findall(r'\s+\[(.*)]\s+', str(tag_content))[0]

        return fix_sup(phonemes)
    except:
        return phonemes

    
def fix_sup(phonemes):
    regex_sups = re.findall(r'\w<sup>\w</sup>', phonemes)

    for sup in regex_sups:
        sup_letters = re.findall(r'(\w)<sup>(\w)</sup>', sup)[0]
        phonemes = phonemes.replace(sup, "{%s^%s}" % sup_letters)
    
    return phonemes

In [8]:
tmp = parse_goroh("вживати")
tmp

'ўж{и^е}ва́т{и^е}'

In [9]:
def split_phonemes(phonemes):
    res = []
    complex_phone = False
    phone_candidate = ''

    for i, letter in enumerate(phonemes):

        if letter == '{':
            complex_phone = True
            phone_candidate += letter
            continue
        elif letter == '}':
            complex_phone = False
            phone_candidate += letter
            res.append(phone_candidate)
            phone_candidate = ''
            continue
        # define stressed letter
        elif ord(letter) == 769: 
            res[-1] = res[-1] + letter
        # define softed letter
        elif ord(letter) in (96, 8217):
            res[-1] = res[-1] + "'"
        # define extended letter
        elif ord(letter) == 58:
            res[-1] = res[-1] + letter
        else:
            if complex_phone:
                phone_candidate += letter
            else:
                res.append(letter)
    return res

In [10]:
vocab = []

with open('../data/dict_corp_vis.txt', 'r') as dict_file:
    for line in dict_file:
        if not line.startswith(" "):
            vocab.append(line)
vocab = [word.split()[0] for word in vocab]
filtered_vocab = list(filter(lambda x: len(x)>2, vocab))

print(f'Number of words in vocabulary: {len(vocab)}')
print(f'Number of words after filtering short words: {len(filtered_vocab)}')

Number of words in vocabulary: 246391
Number of words after filtering short words: 246184


In [11]:
def gentle_parsing(word):
    sleep(0.2)
    phonemes = split_phonemes(parse_goroh(word))
    return (word, phonemes)

In [12]:
batch_size = 1024
# num_batches = int(len(filtered_vocab) / batch_size)
num_batches = 1

In [13]:
parsing_res = []

for batch_num in tqdm(range(num_batches+1)):
    tmp_res = process_map(gentle_parsing, 
                          filtered_vocab[batch_size*(batch_num):batch_size*(batch_num+1)], 
                          max_workers=12)
    if len(parsing_res) == 0:
        parsing_res = tmp_res
    else:
        parsing_res.extend(tmp_res)
    sleep(5)

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:40<00:40, 41.00s/it]

100%|██████████| 2/2 [01:16<00:00, 38.07s/it]


In [14]:
phone_res = [(word, phones) for word, phones in parsing_res if len(phones) > 0]

In [15]:
phone_data = [{"word": word, "phones": phones} for word, phones in phone_res]

In [16]:
phone_df = pd.DataFrame(phone_data)

phone_df.sample(10)

,word,phones
1261,Адамівка,"[а, д, а́, м', і, ў, к, а]"
153,абрикосівка,"[а, б, р, {и^е}, к, о́, с', і, ў, к, а]"
1332,Аделінин,"[а, д, {е^и}, л', і́, н, {и^е}, н]"
250,Абуладзе,"[а, б, у, л, а́, {дз}, {е^и}]"
881,авторитарно-бюрократичний,"[а, ў, т, о, р, {и^е}, т, а́, р, н, о, б', у, ..."
395,авіазаводський,"[а, в', і, а, з, а, в, о, {дз}', к, и́, й]"
751,автометричний,"[а, ў, т, о, м, {е^и}, т, р, и́, ч, н, и, й]"
603,автоген,"[а, ў, т, о, г, е́, н]"
108,абордувавши,"[а, б, о, р, д, у, в, а́, ў, ш, {и^е}]"
1315,адвокатуючи,"[а, д, в, о, к, а, т, у́, й, у, ч, {и^е}]"


In [17]:
print(phone_df.sample(1).values)

[['адрон' list(['а', 'д', 'р', 'о́', 'н'])]]


In [18]:
def json2gzip(data, path):
    if not path.endswith(".json.gz"):
        path += ".json.gz"
    json_str = json.dumps(data) + "\n"
    json_bytes = json_str.encode('utf-8')

    with gzip.GzipFile(path, 'w') as fout:
        fout.write(json_bytes)     

def gzip2json(path):
    with gzip.GzipFile(path, 'r') as fin:
        data = json.loads(fin.read().decode('utf-8'))
    return data

In [19]:
json2gzip(phone_data, '../data/phone_data.json.gz')

In [20]:
phone_data = gzip2json('../data/phone_data.json.gz')

In [21]:
phone_data[42]

{'word': 'абеткуючи',
 'phones': ['а', 'б', '{е^и}', 'т', 'к', 'у́', 'й', 'у', 'ч', '{и^е}']}